## Лабораторная работа


### Форматирование строк

1\. Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в виде `pd.DataFrame` `recipes` При помощи форматирования строк выведите информацию об id рецепта и времени выполнения 5 случайных рецептов в виде таблицы следующего вида:

    
    |      id      |  minutes  |
    |--------------------------|
    |    61178     |    65     |
    |    202352    |    80     |
    |    364322    |    150    |
    |    26177     |    20     |
    |    224785    |    35     |
    
Обратите внимание, что ширина столбцов заранее неизвестна и должна рассчитываться динамически, в зависимости от тех данных, которые были выбраны. 

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/recipes_sample.csv')
df_5 = df.sample(5)

FileNotFoundError: [Errno 2] No such file or directory: 'data/recipes_sample.csv'

In [ ]:
max_len_id = max(df_5['id'].astype(str).apply(len))
max_len_minutes = max(df_5['minutes'].astype(str).apply(len))

id = 'id'
minutes = 'minutes'

print(f'|{id:^{max_len_id + 8}}|{minutes:^{max_len_minutes + 8}}|')
print('|' + '-'*(8 + 9 + len(id) + len(minutes))+ '|')

for index, row in df_5.iterrows():
  id = row['id']
  minutes = row['minutes']
  print(f'|{id:^{max_len_id + 8}}|{minutes:^{max_len_minutes + 8}}|')

2\. Напишите функцию `show_info`, которая по данным о рецепте создает строку (в смысле объекта python) с описанием следующего вида:

```
"Название Из Нескольких Слов"

1. Шаг 1
2. Шаг 2
----------
Автор: contributor_id
Среднее время приготовления: minutes минут
```

    
Данные для создания строки получите из файлов `recipes_sample.csv` (__ЛР2__) и `steps_sample.xml` (__ЛР3__). 
Вызовите данную функцию для рецепта с id `170895` и выведите (через `print`) полученную строку на экран.

In [ ]:
df_recipes = pd.read_csv('data/recipes_sample.csv')

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
with open('data/steps_sample.xml') as f:
  ab = BeautifulSoup(f, 'xml')

steps_dict = {}

for recipe in ab.find_all('recipe'):
  recipe_id = int(recipe.id.text)
  steps = [step.text for step in recipe.find_all('step')]
  steps_dict[recipe_id] = steps

In [ ]:

row = df_recipes[df_recipes['id'] == 170895]

name = row['name'].iloc[0]
minutes = row['minutes'].iloc[0]
author_id = row['contributor_id'].iloc[0]
steps = steps_dict[170895]



In [ ]:
def show_info(name: str, steps: list, minutes: int, author_id: int) -> str:
  s = f'"{name.title()}"\n\n'

  for index, step in enumerate(steps):
    s += f'{index+1}. {step.capitalize()}\n'
  s += '-' * 10 + '\n'
  s += f'Автор: {author_id}\n'
  s += f'Среднее время приготовления: {minutes} минут\n'


  return s


In [ ]:
print(show_info(name, steps, minutes, author_id))

In [ ]:
# noinspection Assert
assert (
        show_info(
            name="george s at the cove black bean soup",
            steps=[
                "clean the leeks and discard the dark green portions",
                "cut the leeks lengthwise then into one-inch pieces",
                "melt the butter in a medium skillet , med",
            ],
            minutes=90,
            author_id=35193,
        )
        == '"George S At The Cove Black Bean Soup"\n\n1. Clean the leeks and discard the dark green portions\n2. Cut the leeks lengthwise then into one-inch pieces\n3. Melt the butter in a medium skillet , med\n----------\nАвтор: 35193\nСреднее время приготовления: 90 минут\n'
)

## Работа с регулярными выражениями

3\. Напишите регулярное выражение, которое ищет следующий паттерн в строке: число (1 цифра или более), затем пробел, затем слова: hour или hours или minute или minutes. Произведите поиск по данному регулярному выражению в каждом шаге рецепта с id 25082. Выведите на экран все непустые результаты, найденные по данному шаблону.

In [ ]:
import re

for index, step in enumerate(steps_dict[25082]):
  res = re.findall(r"[1-9][0-9]+ hour[s]?|[1-9][0-9]+ minute[s]?", step)
  if len(res) != 0 :
    print(f'Шаг {index+1}: {res}')
  

4\. Напишите регулярное выражение, которое ищет шаблон вида "this..., but" _в начале строки_ . Между словом "this" и частью ", but" может находиться произвольное число букв, цифр, знаков подчеркивания и пробелов. Никаких других символов вместо многоточия быть не может. Пробел между запятой и словом "but" может присутствовать или отсутствовать.

Используя строковые методы `pd.Series`, выясните, для каких рецептов данный шаблон содержится в тексте описания. Выведите на экран количество таких рецептов и 3 примера подходящих описаний (текст описания должен быть виден на экране полностью).

In [ ]:
df_recipes['description'].isna().sum()

In [ ]:
df_recipes['description'].fillna(' ', inplace=True)

In [ ]:
df_4 = df[df_recipes['description'].str.contains('^this[\w\d\s]+,[ ]?but', regex=True)]
print('Количество подходящих описаний:', df_4.shape[0])

In [ ]:
pd.set_option('max_colwidth', int(df_4['description'].apply(len).max() + 10))
df_4['description'].sample(3)